<b>Sell Lemonade like a Pro with Gemini and Google Sheets</b>

Are you an aspiring Lemonade baron? Or maybe trying to sell something else to your friends and family? Use Gemini to create pitches customized to the people you think who most love lemons.

In [1]:
#@title Authenticate and grant Colab access to your spreadsheets for this session

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
#@title Configure Gemini API key

#Access your Gemini API key

import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = 'GOOGLE_API_KEY'  # @param {type: "string"}

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}')
   raise e
except userdata.NotebookAccessError as e:
  print(f'You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

model = genai.GenerativeModel('gemini-pro')

#Create a spreadsheet

In [drive.google.com](drive.google.com) create a spreadsheet with the data Gemini needs: name, relationship, and context about that person and your relationship to help personalize a message.

In [5]:
#@title Enter the url of a spreadsheet to import and subject to sell

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1E28W9EUvGn90zui8tG13PtnTaVbC_4VkbY1_NEaA2Ec/edit#gid=0" #@param {type:"string"}
sales_topic = "Lemonade" #@param {type:"string"}

worksheet = gc.open_by_url(spreadsheet_url).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows[0:5])

[['Name', 'Relationship', 'Context'], ['Corrie', 'Coworker', 'Loves sewing. Unclear love of lemonade.'], ['Rishi', 'Neighbor', 'Loves baked goods. Incredible chef.'], ['Bill', 'Town mayor', 'Owes me a favor for not installing those speed bumps yet.']]


In [ ]:
#@title Use Gemini to suggest what to say to pitch them

rows[0].append('Pitch')

#skip the title row
for row in rows[1:]:
  prompt = "I'm selling %s. Can you suggest a short paragraph of text for how I might best pitch them on buying? I'm writing a note to %s my %s. Here's some context about them: %s." % (sales_topic, row[0], row[1], row[2])

  response = model.generate_content(prompt)
  row.append(response.text)

#print out the results nicely
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(''.join([row[3]+'\n\n' for row in rows[1:]]))

> Corrie, I'm offering you a delicious and refreshing treat - my lemonade! You will surely love its tangy and sweet flavor, which I made with fresh lemons and pure water. Not only is it a great way to cool off on a hot day, but it's also a perfect match for your sewing hobby. Imagine sipping on my lemonade while you create your beautiful masterpieces - it will enhance your creativity and make the whole experience even more enjoyable. Don't miss out on this opportunity to indulge in a delightful drink that will complement your passions. Try it now and let the taste of my lemonade inspire your next sewing project!
> 
> Hi Rishi, 
> 
> I'm back again with my homemade lemonade, and this time, I've got a special treat for you - a batch that's been infused with a hint of lemon zest and fresh mint. I know you're an incredible chef with a sweet tooth, so I thought you might enjoy this refreshing and slightly tangy twist on a classic.
> 
> Would you care to try a glass? I'm sure it'll be a great complement to your culinary creations.
> 
> Dear Mayor Bill,
> 
> I would love to sell you some of my delicious, refreshing lemonade! Given that you owe me a favor for not installing those speed bumps, I imagine this should solidify our future business relationship. Not only is my lemonade a thirst-quenching treat, but it also supports our local community. Please support our local youth entrepreneurship and indulge in a cup of my refreshing lemonade! 
> 
> Sincerely,
> 
> [Your Name]
> 


#(OPTIONAL) Write the notes back to your spreadsheet

This only works for a spreadsheet you own, not the example sheet!

In [7]:
#@title Write the notes back to your spreadsheet

#this chooses which column to write to
col = len(rows[0])
start_cell = chr(col + 64) + '1'

#write the column
try:
  worksheet.update(start_cell, [[row[3]] for row in rows])
except Exception as e:
  print('There was an error writing to your spreadsheet. Make sure you have permissions to edit the spreadsheet.')
  raise e

#The notes are in!

Go check your spreadsheet in Drive - there's a new column with the pitches Gemini created. Good luck with your lemonade sales!